### Import Required Libraries

In [1]:
import sys
import os
import numpy
import nltk
import torch
import textwrap
from dotenv import load_dotenv
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.vectorstores import FAISS


### Download supporting package from nltk

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kathi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Create environment for api

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

### Pass the URL and Extract the data from the URL's

In [5]:
URLs = [
    "https://medium.com/@malavikagowthaman/the-ultimate-roadmap-to-becoming-an-ai-engineer-f8b262c6ecb6",
    "https://www.linkedin.com/pulse/introduction-gen-ai-malavika-gowthaman-nlxmc/?trackingId=hzYLZuX7S2qy%2FbNR1kbhZg%3D%3D",
    "https://www.linkedin.com/pulse/end-pipeline-generative-ai-malavika-gowthaman-sckwc/?trackingId=hzYLZuX7S2qy%2FbNR1kbhZg%3D%3D",
    "https://www.linkedin.com/pulse/essential-text-preprocessing-techniques-generative-ai-gowthaman-o34lc/?trackingId=hzYLZuX7S2qy%2FbNR1kbhZg%3D%3D"
]

In [6]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()
data

[Document(metadata={'source': 'https://medium.com/@malavikagowthaman/the-ultimate-roadmap-to-becoming-an-ai-engineer-f8b262c6ecb6'}, page_content='Open in app\n\nSign in\n\nWrite\n\nSign in\n\nThe Ultimate Roadmap to Becoming an AI Engineer\n\nMalavikagowthaman\n\nFollow\n\n5 min read\n\n2 days ago\n\n--\n\nWhy AI Engineering? AI engineering is at the forefront of technology innovation, powering advancements in fields like healthcare, finance, robotics, and autonomous systems. The demand for skilled AI engineers is growing rapidly, making it an exciting and rewarding career path. This roadmap offers a detailed, step-by-step approach to the skills, tools, and experience you’ll need to succeed in AI engineering.\n\nStep 1: Understand AI Engineering Basics\n\nGain an overview of the field and understand where AI fits into the technology landscape.\n\nKey Concepts: AI vs. Machine Learning vs. Deep Learning, types of AI (Narrow, General, Super AI)\n\nSpecializations: Computer Vision, Natura

In [7]:
len(data)

4

### Spliting data's into chuncks

In [8]:
text_splitter = CharacterTextSplitter(separator="\n",
                                      chunk_size = 1000,
                                      chunk_overlap = 100)

In [9]:
text_chunk = text_splitter.split_documents(data)

In [10]:
text_chunk

[Document(metadata={'source': 'https://medium.com/@malavikagowthaman/the-ultimate-roadmap-to-becoming-an-ai-engineer-f8b262c6ecb6'}, page_content='Open in app\nSign in\nWrite\nSign in\nThe Ultimate Roadmap to Becoming an AI Engineer\nMalavikagowthaman\nFollow\n5 min read\n2 days ago\n--\nWhy AI Engineering? AI engineering is at the forefront of technology innovation, powering advancements in fields like healthcare, finance, robotics, and autonomous systems. The demand for skilled AI engineers is growing rapidly, making it an exciting and rewarding career path. This roadmap offers a detailed, step-by-step approach to the skills, tools, and experience you’ll need to succeed in AI engineering.\nStep 1: Understand AI Engineering Basics\nGain an overview of the field and understand where AI fits into the technology landscape.\nKey Concepts: AI vs. Machine Learning vs. Deep Learning, types of AI (Narrow, General, Super AI)\nSpecializations: Computer Vision, Natural Language Processing, Robot

In [11]:
text_chunk[0]

Document(metadata={'source': 'https://medium.com/@malavikagowthaman/the-ultimate-roadmap-to-becoming-an-ai-engineer-f8b262c6ecb6'}, page_content='Open in app\nSign in\nWrite\nSign in\nThe Ultimate Roadmap to Becoming an AI Engineer\nMalavikagowthaman\nFollow\n5 min read\n2 days ago\n--\nWhy AI Engineering? AI engineering is at the forefront of technology innovation, powering advancements in fields like healthcare, finance, robotics, and autonomous systems. The demand for skilled AI engineers is growing rapidly, making it an exciting and rewarding career path. This roadmap offers a detailed, step-by-step approach to the skills, tools, and experience you’ll need to succeed in AI engineering.\nStep 1: Understand AI Engineering Basics\nGain an overview of the field and understand where AI fits into the technology landscape.\nKey Concepts: AI vs. Machine Learning vs. Deep Learning, types of AI (Narrow, General, Super AI)\nSpecializations: Computer Vision, Natural Language Processing, Roboti

### Embedding model

In [12]:
embeddings = OpenAIEmbeddings()
# embeddings

C:\Users\kathi\AppData\Local\Temp\ipykernel_37452\2080068311.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [13]:
query = embeddings.embed_query("Welcome")
query

[-0.004538055611952574,
 0.0038519220832092236,
 -0.008214455283675691,
 -0.02782830286372135,
 -0.03444071564476635,
 0.00970160983090374,
 -0.006542204095847334,
 -0.004943353235752024,
 -0.00923567751819018,
 -0.0013012603452398665,
 0.020832930231334205,
 -0.019530872324706464,
 -0.012369552906931305,
 -0.01833093580333984,
 0.016288493196956053,
 0.0029711179175820105,
 0.04595499589653897,
 -0.010039889480718312,
 -0.0006586084640511199,
 -0.018969200980479963,
 -0.03347055317155982,
 -0.0007132598127416444,
 0.0032870586307860404,
 -0.0005876016353889613,
 -0.019786177650504437,
 -0.021943510074936057,
 -0.0043050894555957935,
 -0.012426995953310033,
 0.009025050531274592,
 -0.007563426330361822,
 0.0031258969808946155,
 -0.013275887186043148,
 -0.021764796719406516,
 -0.01593106462325178,
 -0.0168629292486789,
 -0.018305406388347156,
 -0.011769584646247992,
 -0.000556486263882736,
 0.022326468063633018,
 0.0019451089177563948,
 0.022849845285370495,
 -0.005115684237533396,
 0.0

In [14]:
len(query)

1536

### COnvert the text chunks into embeedings

In [15]:
vectorstores = FAISS.from_documents(text_chunk,embeddings)
vectorstores

### Create LLM Wrapper

In [16]:
llm = ChatOpenAI()

C:\Users\kathi\AppData\Local\Temp\ipykernel_37452\44520985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [17]:
llm.predict("Welcome message for user")

C:\Users\kathi\AppData\Local\Temp\ipykernel_37452\3703220341.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Welcome message for user")


'Welcome! Thank you for joining us. We are thrilled to have you here and look forward to connecting with you. If you have any questions or need assistance, feel free to reach out. Enjoy your time with us!'

### Initialize the Retrieval QA with source chain

In [18]:
chain = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,
    retriever = vectorstores.as_retriever()
)

In [19]:
result = chain({"question":"What is Gen AI?"}, return_only_outputs= True)

C:\Users\kathi\AppData\Local\Temp\ipykernel_37452\3644462141.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question":"What is Gen AI?"}, return_only_outputs= True)


In [20]:
result['answer']

'Gen AI, or Generative AI, is a type of artificial intelligence that can create new content, like writing, images, music, or code, through understanding and mimicking human creativity and intelligence.\n'

In [21]:
result = chain({"question":"What is AI Engineering?"}, return_only_outputs= True)

In [22]:
result['answer']

'AI Engineering is the role of an AI Engineer, who works with AI models and is grounded in math, programming skills, and data processing. It involves utilizing programming languages such as Python, SQL, JavaScript, and C++ for data extraction and manipulation, as well as proficiency in data processing and analysis.\n'

In [23]:
result = chain({"question":"Difference between LLM and GenAI"}, return_only_outputs= True)

In [24]:
result['answer']

'LLM is a type of generative AI that focuses specifically on generating text, while Gen AI is a broader term that encompasses various types of generative AI, including LLM, but also extends to generating images, music, code, and other forms of content.\n'

In [25]:
while True: 
    query = input(f"Prompt: ")
    # print("Question: ",query)
    if query == 'q':
        print("Exiting")
        sys.exit()
        q
    if query == "":
        continue
    result=chain({'question':query})
    print("Question: ",query)
    print(f"Answer: " +result["answer"])

Question:  what is LLM?
Answer: LLM can stand for Large Language Model, Master of Laws, or Lemmatization.

Question:  Whar is Genai
Answer: Gen AI stands for Generative AI, which is a type of artificial intelligence that can create new content. 

Exiting


SystemExit: 

c:\Users\kathi\anaconda3\envs\customchatbot\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
